# Customer Segmentation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import dendrogram

# Introduction to Clustering

Clustering is the task of dividing the unlabeled data or data points into different clusters such that similar data points fall  \
in the same cluster than those which differ from the others. In simple words, the aim of the clustering process is to segregate groups with similar traits and assign them into clusters.

<img src="https://i.stack.imgur.com/cIDB3.png">

## Importing the Dataset

In [ ]:
data = pd.read_csv('../input/customer-clustering/segmentation data.csv')

In [ ]:
data.shape

In [ ]:
data.drop(['ID'], inplace=True, axis=1)

In [ ]:
data.head(10)

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

## Exploratory Data Analysis

In [ ]:
plt.figure(figsize=(21,15))

plt.subplot2grid((2,2), (0,0))
box1 = sns.boxplot(y=data.Age)
plt.title("Age")

plt.subplot2grid((2,2), (0,1))
box2 = sns.boxplot(y=data.Income)
plt.title("Income")

plt.show()

In [ ]:
data.Age.describe()

In [ ]:
data.Income.describe()

### Inferences
- Mean age is approximately 36 years. Max is 76 meanwhile Min is 18.
- Mean income is 121k. Max is 310k meanwhile Min is 36k.

### Proportion of data values in each feature

In [ ]:
plt.figure(figsize=(21,15))

plt.subplot2grid((3,3), (0,0))
sns.histplot(data.Sex.astype(str), stat='proportion')

plt.subplot2grid((3,3), (0,1))
sns.histplot(data['Marital status'].astype(str), stat='proportion')

plt.subplot2grid((3,3), (0,2))
sns.histplot(data.Education.astype(str).sort_values(), stat='proportion')

plt.subplot2grid((3,3), (1,0))
sns.histplot(data.Occupation.astype(str).sort_values(), stat='proportion')

plt.subplot2grid((3,3), (1,1))
sns.histplot(data['Settlement size'].astype(str).sort_values(), stat='proportion')

plt.show()

# Standardizing the Data

In [ ]:
ss = StandardScaler()
data = pd.DataFrame(ss.fit_transform(data))

# K Means Model

K-means clustering is a popular unsupervised machine learning algorithm used to group data points with similar characteristics into K number of clusters. It works by first randomly assigning K number of centroids to the data points, and then iteratively adjusting the centroids to minimize the sum of squared distances between the data points and their assigned centroid. This process continues until convergence or until a maximum number of iterations is reached.

<img src="https://av-eks-blogoptimized.s3.amazonaws.com/40672tb2.png">

## Determining number of clusters : Elbow Method

The elbow method is a technique used to determine the optimal number of clusters in a dataset for k-means clustering. The method works by plotting the Within Cluster Sum of Squares (WCSS) between each data point and its assigned centroid for different values of K.

As K increases, the WCSS tends to decrease, since with more clusters, each data point is likely to be closer to its assigned centroid. However, beyond a certain point, the improvement in SSE becomes less significant, and the plot of WCSS versus K starts to form an elbow shape. For this, we will train multiple K means model with clusters ranging from 1-10.

In [ ]:
wcss = {'wcss_score':[], 'no_of_clusters':[]}
for i in range(1,11):
    kmeans = KMeans(i, random_state=0)
    kmeans.fit(data)
    wcss['wcss_score'].append(kmeans.inertia_)
    wcss['no_of_clusters'].append(i)
wcss_df = pd.DataFrame(wcss)

In [ ]:
wcss_df.head(10)

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(wcss_df.no_of_clusters, wcss_df.wcss_score, marker='o')
plt.title("Elbow Method to determine number of clusters(K)")
plt.show()

### Inference
At 4 clusters, we can see that the decrease in WCSS starts declining. Therefore, the number of clusters within this dataset is **4**
* K = 4

In [ ]:
kmeans_final = KMeans(n_clusters=4, random_state=0, init='k-means++')
classlabels = kmeans_final.fit_predict(data)

In [ ]:
data['classlabels'] = classlabels
data.classlabels = data.classlabels.astype(str)
data = data.sort_values('classlabels')

In [ ]:
plt.figure(figsize=(14,10))
sns.histplot(data.classlabels)
plt.show()

# Evaluating our Model : Silhouette Score

The silhouette score is a metric used to evaluate the quality of clustering in a dataset. \
It measures how similar a data point is to its own cluster compared to other clusters.

The silhouette score ranges from -1 to 1, with a higher score indicating better clustering performance. \
A score of 1 indicates that a data point is very similar to its own cluster and very dissimilar to other clusters, while a score of -1 indicates the opposite.

To calculate the silhouette score for a cluster, the average distance between a data point and all other points in the same cluster is first computed. \
The average distance between the data point and all points in the nearest neighboring cluster is then calculated. The silhouette score for the data point is then given by the difference between these two values, divided by the maximum of the two.

In [ ]:
score = silhouette_score(data, kmeans_final.labels_, random_state=0)
print(f"Silhouette score: {score:0.3f} ~ 0")

Silhouette score of 0 means our model did not work very well. The worse could be -1, but the best can go upto 1.

# Hierarchical clustering - Agglomerative

Hierarchical clustering groups similar objects into a dendrogram. It merges similar clusters iteratively, starting with each data point as a separate cluster.  \
This creates a tree-like structure that shows the relationships between clusters and their hierarchy.

The dendrogram from hierarchical clustering reveals the hierarchy of clusters at different levels, highlighting natural groupings in the data.  \
It provides a visual representation of the relationships between clusters, helping to identify patterns and outliers, making it a useful tool for exploratory data analysis. For example:

Let’s say we have the below points and we want to cluster them into groups:  \
<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/05/Screenshot-from-2019-05-15-13-10-32.png" alt="">

We can assign each of these points to a separate cluster:   \
<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/05/Screenshot-from-2019-05-15-13-11-28.png" alt="">

Now, based on the similarity of these clusters, we can combine the most similar clusters together and repeat this process until only a single cluster is left:   \
<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/05/Screenshot-from-2019-05-15-13-12-35.png" alt="">

We are essentially building a hierarchy of clusters. That’s why this algorithm is called hierarchical clustering. This type of Hierarchical clustering is called **Agglomerative hierarchical clustering**. \
We assign each point to an individual cluster in this technique. Then, at each iteration, we merge the closest pair of clusters and repeat this step until only a single cluster is left:   \
<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2019/05/Screenshot-from-2019-05-15-13-31-06.png" alt="">

To improve the clustering model, we move to hierarchical clustering

In [ ]:
new_data = data.drop(['classlabels'], axis=1)

### Distances and Linkages

Distances and linkages are two key components of hierarchical clustering
* **Distances**: Distances refer to the measure of dissimilarity or similarity between two data points in a dataset.
* **Linkages**: Linkages refer to the method used to compute the distance between clusters during the clustering process.

With multiple computation options for both distance and linkage in clusters, we calculate the silhouette score for all permutations

In [ ]:
## function to compute scores for all permutations
def s_score(distance, linkage):
    agc = AgglomerativeClustering(n_clusters=4, affinity=distance, linkage=linkage)
    agc.fit_predict(new_data)
    score = silhouette_score(new_data, agc.labels_, random_state=0)
    return score

In [ ]:
distances = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
linkages = ['ward', 'complete', 'average', 'single']

In [ ]:
scoring = {'dist':[], 'link':[], 'sScore':[]}
for i in distances:
    for j in linkages:
        try:
            score = s_score(i, j)
            scoring['dist'].append(i)
            scoring['link'].append(j)
            scoring['sScore'].append(score)
        except:
            scoring['dist'].append(i)
            scoring['link'].append(j)
            scoring['sScore'].append(np.nan)
scoringDf = pd.DataFrame(scoring)


**NOTE**: We put this process in try-except block since 'ward' only works with 'euclidean' distance. We can now find the best permutation.

In [ ]:
scoringDf.dropna(axis=0, inplace=True)

In [ ]:
final_result = scoringDf[scoringDf['sScore'] == max(scoringDf['sScore'])]
final_result

# DBScan Clustering

**DBSCAN** stands for **D**ensity-**B**ased **S**patial **C**lustering of **A**pplications with **N**oise.

It groups ‘densely grouped’ data points into a single cluster. It can identify clusters in large spatial datasets by looking at the local density of the data points.  \
The most exciting feature of DBSCAN clustering is that it is robust to outliers. It also does not require the number of clusters to be told beforehand, unlike K-Means, where we have to specify the number of centroids.

DBSCAN requires only two parameters: epsilon and minPoints. Epsilon is the radius of the circle to be created around each data point to check the density \
and minPoints is the minimum number of data points required inside that circle for that data point to be classified as a Core point

<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2020/03/db26.png" alt=""> <br>

#### This is what it looks like:  

<img src="https://cdn.analyticsvidhya.com/wp-content/uploads/2020/03/db12.png" alt=""> <br>

In [ ]:
dbs = DBSCAN(eps=2, min_samples=10)
dbs.fit_predict(new_data)
score = silhouette_score(new_data, dbs.labels_, random_state=0)
print(f"Silhouette score: {score:0.3f}")